## In this notebook...

We will analyze the [Pushshift Reddit dataset](https://arxiv.org/pdf/2001.08435.pdf) to be used for the project and then we will run a SageMaker Processing Job to filter out the comments and submissions from subreddits of interest. The filtered data will be stored in your account's s3 bucket and it is this filtered data that you will be using for your project.

## Setup
We need an available Java installation to run pyspark. The easiest way to do this is to install JDK and set the proper paths using conda

In [8]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.2.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    ------------------------------------------------------------
                                           Total:         158 KB

The following packages will be UPDATED:

  certifi                         2023.7.22-py310h06a4308_0 --> 2023.11.17-py310h06a4308_0 



                                                                        

In [5]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

3.2.0


## Process S3 data with SageMaker Processing Job `PySparkProcessor`

We are going to move the above processing code in a Python file and then submit that file to SageMaker Processing Job's [`PySparkProcessor`](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html#pysparkprocessor).

In [5]:
!mkdir -p ./code

In [9]:
%%writefile ./code/process.py

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path", type=str, help="Path of dataset in S3")    
    parser.add_argument("--s3_output_bucket", type=str, help="s3 output bucket")
    parser.add_argument("--s3_output_prefix", type=str, help="s3 output prefix")
    parser.add_argument("--col_name_for_filtering", type=str, help="Name of the column to filter")
    parser.add_argument("--values_to_keep", type=str, help="comma separated list of values to keep in the filtered set")
    args = parser.parse_args()

    spark = SparkSession.builder.appName("PySparkApp").getOrCreate()
    logger.info(f"spark version = {spark.version}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

   
    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path}")
    df = spark.read.parquet(args.s3_dataset_path, header=True)
    logger.info(f"finished reading files...")
    

    
    # filter the dataframe to only keep the values of interest
    vals = [s.strip() for s in args.values_to_keep.split(",")]
    df_filtered = df.where(col(args.col_name_for_filtering).isin(vals))
    
    # save the filtered dataframes so that these files can now be used for future analysis
    s3_path = f"s3://{args.s3_output_bucket}/{args.s3_output_prefix}"
    logger.info(f"going to write data for {vals} in {s3_path}")
    logger.info(f"shape of the df_filtered dataframe is {df_filtered.count():,}x{len(df_filtered.columns)}")
    df_filtered.write.mode("overwrite").parquet(s3_path)
    
    logger.info(f"all done...")
    
if __name__ == "__main__":
    main()

Overwriting ./code/process.py


In [10]:
bucket_name = "project-group34"
# !aws s3 mb s3://{bucket_name}

In [11]:
%%time
import time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    framework_version="3.3",
    role=role,
    instance_count=8,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=3600,
)

# s3 paths
session = sagemaker.Session()
bucket = bucket_name
output_prefix_logs = f"spark_logs"

configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {"spark.executor.memory": "12g", "spark.executor.cores": "4"},
    }
]

# the dataset contains data for these 3 years
year_list = [2021,2022,2023]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.73 s, sys: 501 ms, total: 2.24 s
Wall time: 1.95 s


## COMMENT DATA EXTRACTION

### For subreddits - r/MovieSuggestions, r/televisionsuggestions, r/Animesuggest

In [9]:
%%time
for yyyy in year_list:
    print(f"going to filter comments data for year={yyyy}")
    s3_dataset_path_commments = f"s3://bigdatateaching/reddit-parquet/comments/year={yyyy}/month=*/*.parquet" # "s3a://bigdatateaching/reddit/parquet/comments/yyyy=*/mm=*/*comments*.parquet"
    output_prefix_data_comments = f"project/comments/yyyy={yyyy}"
    col_name_for_filtering = "subreddit"
    subreddits = "MovieSuggestions, televisionsuggestions, Animesuggest"

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/process.py",
        arguments=[
            "--s3_dataset_path",
            s3_dataset_path_commments,
            "--s3_output_bucket",
            bucket,
            "--s3_output_prefix",
            output_prefix_data_comments,
            "--col_name_for_filtering",
            col_name_for_filtering,
            "--values_to_keep",
            subreddits,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

going to filter comments data for year=2021


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-17-19-09-27-774


...............................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-17-19-27-56-444


going to filter comments data for year=2022
................................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-17-19-47-50-664


going to filter comments data for year=2023
..................................................................................................................!CPU times: user 2.5 s, sys: 348 ms, total: 2.85 s
Wall time: 49min 2s


## SUBMISSIONS DATA EXTRACTION

### For subreddits - r/movies, r/television, r/anime

In [16]:
%%time
for yyyy in year_list:
    print(f"going to filter submissions data for year={yyyy}")
    s3_dataset_path_submissions = f"s3://bigdatateaching/reddit-parquet/submissions/year={yyyy}/month=*/*.parquet" # "s3a://bigdatateaching/reddit/parquet/submissions/yyyy=*/mm=*/*submissions*.parquet"
    output_prefix_data_submissions = f"project/submissions/yyyy={yyyy}"
    col_name_for_filtering = "subreddit"
    subreddits = "movies, television, anime"

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/process.py",
        arguments=[
             "--s3_dataset_path",
            s3_dataset_path_submissions,
            "--s3_output_bucket",
            bucket,
            "--s3_output_prefix",
            output_prefix_data_submissions,
            "--col_name_for_filtering",
            col_name_for_filtering,
            "--values_to_keep",
            subreddits,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

going to filter submissions data for year=2021


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-16-21-13-29-063


................................................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-16-21-34-44-397


going to filter submissions data for year=2022
.........................................................................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-16-21-58-05-649


going to filter submissions data for year=2023
....................................................................................................................................!CPU times: user 3.42 s, sys: 464 ms, total: 3.88 s
Wall time: 56min 47s


### For subreddits - r/MovieSuggestions, r/televisionsuggestions, r/Animesuggest

In [12]:
%%time
for yyyy in year_list:
    print(f"going to filter submissions data for year={yyyy}")
    s3_dataset_path_submissions = f"s3://bigdatateaching/reddit-parquet/submissions/year={yyyy}/month=*/*.parquet" # "s3a://bigdatateaching/reddit/parquet/submissions/yyyy=*/mm=*/*submissions*.parquet"
    output_prefix_data_submissions = f"project/submissions/suggestions/yyyy={yyyy}"
    col_name_for_filtering = "subreddit"
    subreddits = "MovieSuggestions, televisionsuggestions, Animesuggest"

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/process.py",
        arguments=[
             "--s3_dataset_path",
            s3_dataset_path_submissions,
            "--s3_output_bucket",
            bucket,
            "--s3_output_prefix",
            output_prefix_data_submissions,
            "--col_name_for_filtering",
            col_name_for_filtering,
            "--values_to_keep",
            subreddits,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

going to filter submissions data for year=2021


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-20-19-39-33-402


...................................................

KeyboardInterrupt: 

## Read the filtered data

Now that we have filtered the data to only keep submissions and comments from subreddits of interest. Let us read data from the s3 path where we saved the filtered data.

In [10]:
%%time
import sagemaker
session = sagemaker.Session()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
#s3_path = "s3a://sagemaker-us-east-1-038932893404/project/comments/yyyy=2021/part-00000-90796409-5783-4705-92c0-27c27eda8c4c-c000.snappy.parquet"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://project-group34/project/comments/yyyy=*


23/11/17 20:00:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 32 ms, sys: 41 µs, total: 32 ms
Wall time: 5.8 s


In [11]:
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

shape of the comments dataframe is 2,391,714x21


In [12]:
# check counts (ensuring all needed subreddits exist)
comments.groupBy('subreddit').count().show()

+--------------------+-------+
|           subreddit|  count|
+--------------------+-------+
|televisionsuggest...| 112586|
|        Animesuggest|1025097|
|    MovieSuggestions|1254031|
+--------------------+-------+



In [13]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [14]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc").show()

+----------------+---------------+--------------------+----------+---------+-------+-------------------+
|       subreddit|         author|                body| parent_id|  link_id|     id|        created_utc|
+----------------+---------------+--------------------+----------+---------+-------+-------------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|t3_m3ygv3|gqscelh|2021-03-13 10:15:52|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|t3_m3ygv3|gqscf1z|2021-03-13 10:16:05|
|    Animesuggest|      [deleted]|           [deleted]| t3_m3vnjl|t3_m3vnjl|gqscjse|2021-03-13 10:18:25|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|t3_m3rw47|gqscl5i|2021-03-13 10:19:07|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| t3_m43dco|t3_m43dco|gqscnr8|2021-03-13 10:20:26|
|MovieSuggestions|   alienstabler|[Holes (2003)](ht...|

In [15]:
%%time
import sagemaker
session = sagemaker.Session()
output_prefix_data_submissions = f"project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://project-group34/project/submissions/yyyy=*
CPU times: user 32.6 ms, sys: 0 ns, total: 32.6 ms
Wall time: 1.14 s


23/11/17 20:14:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [16]:
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

shape of the submissions dataframe is 875,969x68


In [17]:
# check counts (ensuring all needed subreddits exist)
submissions.groupBy('subreddit').count().show()

+----------+------+
| subreddit| count|
+----------+------+
|     anime|404298|
|television| 89586|
|    movies|382085|
+----------+------+



In [18]:
submissions.printSchema()

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

In [19]:
# display a subset of columns
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments").show()

+----------+--------------------+--------------------+--------------------+-------------------+------------+
| subreddit|              author|               title|            selftext|        created_utc|num_comments|
+----------+--------------------+--------------------+--------------------+-------------------+------------+
|television|           Hyhyhyhuh|Is there a websit...|Hey all,  \n\n\nI...|2021-01-27 22:16:47|           9|
|     anime|           [deleted]|Does anyone know ...|           [deleted]|2021-01-27 22:17:30|           3|
|television|          boxermansr|‘Doogie Kameāloha...|                    |2021-01-27 22:17:42|          11|
|    movies|   cheesemaster_3000|4K movies on desk...|           [removed]|2021-01-27 22:18:23|           4|
|     anime|           [deleted]|Where can I buy a...|           [deleted]|2021-01-27 22:19:23|           9|
|     anime|              Mjrbks|Ever suddenly fin...|I never really us...|2021-01-27 22:20:14|           7|
|    movies|       

In [3]:
bucket = "project-group34"

In [16]:
# Copy the files to Sangeetha's, and Samiksha's s3 buckets
## Comments
!aws s3 cp s3://{bucket}/project/comments/ s3://sk2224-projectdata/comments/ --recursive
# !aws s3 cp s3://{bucket}/project/submissions/ s3://sk2224-projectdata/submissions/ --recursive

copy: s3://project-group34/project/comments/yyyy=2021/_SUCCESS to s3://sk2224-projectdata/comments/yyyy=2021/_SUCCESS
copy: s3://project-group34/project/comments/yyyy=2021/part-00000-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet to s3://sk2224-projectdata/comments/yyyy=2021/part-00000-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet
copy: s3://project-group34/project/comments/yyyy=2021/part-00004-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet to s3://sk2224-projectdata/comments/yyyy=2021/part-00004-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet
copy: s3://project-group34/project/comments/yyyy=2021/part-00008-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet to s3://sk2224-projectdata/comments/yyyy=2021/part-00008-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet
copy: s3://project-group34/project/comments/yyyy=2021/part-00002-f41f7a80-c788-4c54-a5b9-2cb7303f54d4-c000.snappy.parquet to s3://sk2224-projectdata/comments/yyyy=2021/part-0